# Importing libraries

In [36]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from keras.layers import Dense, Dropout, Activation , Flatten
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split
from sklearn.exceptions import DataConversionWarning
from sklearn.ensemble import RandomForestClassifier
from tensorflow.keras.backend import clear_session
from sklearn.tree import DecisionTreeClassifier
from keras.callbacks import TensorBoard
from keras.models  import Sequential
from itertools import combinations
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from collections import deque
import multiprocessing as mp
from tensorflow import keras
import tensorflow as tf
import pandas as pd
import numpy as np
import warnings
import logging
import random
import pickle
import time
import copy
import os

logging.getLogger("tensorflow").setLevel(logging.ERROR)
warnings.filterwarnings("ignore")
warnings.filterwarnings(action='ignore', category=DataConversionWarning)

In [37]:
from google.colab import drive
drive.mount("/content/drive")
data_path = "/content/drive/MyDrive/Colab Notebooks/Muawiya/Js_Contana/dynamic-feature-selection/Data/isam data"
root_reslts_path = "/content/drive/MyDrive/Colab Notebooks/Muawiya/Js_Contana/dynamic-feature-selection/Results"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Reading and exploration of data:

In [38]:
def read_preprocess(path, ordered_dataset_path):
    try:
      with open(ordered_dataset_path , 'rb') as data:
        print('loaded data!')
        dataset = pickle.load(data)
    except:
      dataset = pd.read_csv(path)
      dataset = dataset[:20000]
      dataset = dataset.sample(frac=1).reset_index(drop = True)
      for col in dataset.columns:
          if np.max(dataset[col]) > 1000:
              dataset.drop([col],axis = 1, inplace = True)
    return dataset

In [39]:
def ExploreData(dataset):
    cols = dataset.columns
    classes =  np.array(dataset.iloc[:, -1]) #np.array(dataset['Class'])
    Data = np.array(dataset.drop(dataset.columns[-1], axis=1)) #np.array((dataset.drop(['Class'] , axis = 1)))
    print(sum(classes == 0)/classes.shape[0] , sum(classes == 1)/classes.shape[0])
    Buffer_Class1 = np.where(classes == 1)
    return Data, classes, cols, np.array(Buffer_Class1[0])

In [40]:
def save_object(obj, filename,path):
    filename = os.path.join(path,filename)
    with open(filename+".pkl", 'wb') as outp:
        pickle.dump(obj, outp, pickle.HIGHEST_PROTOCOL)
    outp.close()
def load_object(filename,path):
    filename = os.path.join(path,filename)
    with open(filename+".pkl", 'rb') as outp:
        loaded_object = pickle.load(outp)
    outp.close()
    return loaded_object

# Reward Calculation:

In [41]:
class Reward:

    def Entropy(self, pi):
        """
        Defines the (discrete) distribution...
        pi is the probabilties of each value in vec i:
        pi = for every element in vec i: number_of_accurance/len(vec)
        """
        return - np.sum(pi * np.log2(pi+ 0.000000000000001))

    def joint_entropy(self,jp):
        """
        H(XY) = p(x,y) * log(p(x,y))
        """
        return self.Entropy(jp)

    def joint_probability(self, X1, X2):
        df = pd.DataFrame({'X1': X1, 'X2': X2})
        df['X3'] = df.apply(lambda x: '_'.join([str(i) for i in x]), axis=1)
        return df['X3'].value_counts().values / df.shape[0], df['X1'].value_counts() / df.shape[0], df['X2'].value_counts() / df.shape[0]

    def mutual_information(self,X,Y):
        """
        Mutual Information: I(Y;X) = H(Y) - H(Y|X)
        Reference: https://en.wikipedia.org/wiki/Information_gain_in_decision_trees#Formal_definition
        """
        jp, X1_proba, X2_proba = self.joint_probability(X, Y)
        MI = self.Entropy(X1_proba) - self.joint_entropy(jp) + self.Entropy(X2_proba)
        return MI

    def GetMiRedundancy(self, data):
        """
        Redundancy = Sum(mutual information between each pair of features) / Squared SPACE_LENGTH
        Then apply sigmoid to it to transferit to a value [0,1]
        """
        Features = np.arange(0, data.shape[1], 1)
        sum_ = 0
        comb = combinations(Features, 2) #every pair of features.
        for Combination in list(comb):
            sum_ = sum_ +  self.mutual_information(data[:,Combination[0]] , data[:,Combination[1]])

        Rd = sum_/(data.shape[1] ** 2)

        if np.isnan(sum_):
            raise ValueError("Redundancy is nan")

        return Rd

    def GetMiRelevance(self, data , labels):
        """
        Relevance = Sum(mutual information between features and labels.) / SPACE_LENGTH
        Then apply sigmoid to it to transferit to a value [0,1]

        """
        Features = np.arange(0, data.shape[1], 1)
        sum_ = 0
        for feature in Features:
            sum_ = sum_ +  self.mutual_information(data[:,feature] , labels)

        Rv = (sum_/data.shape[1])

        if np.isnan(Rv):
            raise ValueError("Relevance is nan")

        return Rv

In [42]:
def F_score(X, Y):
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
    DT_clf = DecisionTreeClassifier(max_depth = 7)

    DT_clf.fit(X_train,Y_train)
    ypred=DT_clf.predict(X_test) #These are the predicted output values
    fscore = f1_score(Y_test,ypred, average='weighted')
    acc = accuracy_score(Y_test,ypred)
    percision = precision_score(Y_test,ypred, average='weighted')
    recall = recall_score(Y_test,ypred, average='weighted')

    return fscore#, acc, percision, recall


# Enviroment:

## state class

In [43]:
class descriptive_statistics_state:

  def get_state(self, state):
      """
      input: np array of the data
      output: 7,7 descriptive statistics np array.
      """
      pd_state = pd.DataFrame(state)
      """
      the statistics are: (mean, std, min, 25%, 50%, 75%, max)
      """
      return np.array(pd_state.describe().drop(['count']).T.describe().drop(['count']))

## enviroment class:

In [44]:
## the environment: (includes the data, Mutual info arrays, selected space and it's statistical rep).
class FeaturesSubspaceEnvironment:
    def __init__(self,SPACE , selected, Data, classes, OBSERVATION_SPACE_VALUES):
        """
        selected : an integar for the chaging feature number.
        """
        self.selected = selected
        self.Data = Data
        self.classes = classes
        self.OBSERVATION_SPACE_VALUES = OBSERVATION_SPACE_VALUES  # the number of discreptive statistics.
        self.SPACE = SPACE
        self.Reward_model = Reward()
        self.descriptive_statistics_obj = descriptive_statistics_state()

    ACTION_SPACE_SIZE = 2 # enable = 1, disable = 0

    def get_Redundancy(self):
        return self.Reward_model.GetMiRedundancy(self.Get_Data())

    def get_relevance(self):
        return self.Reward_model.GetMiRelevance(self.Get_Data(), self.classes)

    # the movement function (changing the selected feature's decision)
    def step(self, action):
        self.SPACE[self.selected] = action

    def assign_new_stata(self):
        self.State = self.get_statistics_state()

    def get_reward(self):
        self.Redundancy = self.get_Redundancy()
        self.relevance = self.get_relevance()
        return (self.relevance - self.Redundancy)

    def Get_Data(self):
        """
        returns the data after deleting the disabled features.
        """
        Indexes = np.where(self.SPACE == 0)[0] # since the notTriggered features have the value of 0,
        # its gurented that we only use the triggered features
        Data_del = np.delete(self.Data, Indexes, 1)
        return Data_del

    def get_statistics_state(self):
        if sum(self.SPACE) == 0:
          return None

        State = self.descriptive_statistics_obj.get_state(self.Get_Data())
        return State

# Deep reinforcement learning model

In [45]:
DISCOUNT = 0.99
REPLAY_MEMORY_SIZE = 500
# How many last steps to keep for model training
MIN_REPLAY_MEMORY_SIZE = 200 # 10% at of the episodes.
# Minimum number of steps in a memory to start training
MINIBATCH_SIZE = 200
# Terminal states (end of episodes)
MODEL_NAME = "FS"

#Agent class
class DQNAgent:
    def __init__(self, OBSERVATION_SPACE_VALUES):

        self.OBSERVATION_SPACE_VALUES = OBSERVATION_SPACE_VALUES
        # Main model
        self.model = self.create_model()
        # An array with last n steps for training
        self.replay_memory = deque(maxlen=REPLAY_MEMORY_SIZE)

    def create_model(self): #MODEL architecture:
        model = Sequential()
        model.add(Flatten(input_shape=self.OBSERVATION_SPACE_VALUES))
        model.add(Dense(32))
        model.add(Dense(16 , activation='relu'))
        model.add(Dense(2, activation='sigmoid'))
        model.compile(loss="mse", optimizer="adam", metrics=['mse'])

        return model


    # Adds step's data to a memory replay array
    # (observation space, action, reward, new observation space, done)
    def update_replay_memory(self, transition):
        if len(self.replay_memory) >= REPLAY_MEMORY_SIZE:
          self.replay_memory.popleft()
        self.replay_memory.append(transition)


    # Trains main network every step during episode
    def train(self):
        # Start training only if certain number of samples is already saved
        if len(self.replay_memory) < MIN_REPLAY_MEMORY_SIZE:
            return
        # Get a minibatch of random samples from memory replay table
        minibatch = random.sample(self.replay_memory, MINIBATCH_SIZE)

        current_states = np.array([transition[0] for transition in minibatch])#/255
        current_qs_list = self.model.predict(current_states,verbose=0)

        # Get future states from minibatch, then query NN model for Q values
        # When using target network, query it, otherwise main network should be queried
        new_current_states = np.array([transition[3] for transition in minibatch])#/255
        future_qs_list = self.model.predict(new_current_states,verbose=0)
        #qs_list = self.model.predict(current_states,verbose=0)

        X = []
        y = []

        # Now we need to enumerate our batches
        for index, (current_state, action, reward, new_current_state, selectedSubspace, current_Subspace) in enumerate(minibatch):
            max_future_q = np.max(future_qs_list[index])
            new_q = reward + DISCOUNT * max_future_q
            # Update Q value for given state
            current_qs = current_qs_list[index] #a new value na
            current_qs[action] = new_q
            # And append to our training data
            X.append(current_state)
            y.append(current_qs)

        # Fit on all samples as one batch, log only on terminal state
        self.model.fit(np.array(X), np.array(y), batch_size=MINIBATCH_SIZE, verbose=0, shuffle=False)
    # Queries main network for Q values given current observation space (environment state)
    def get_qs(self, state):
        y = self.model.predict(np.array(state).reshape(-1, *state.shape),verbose=0)[0]
        return y



# Helping functions

In [46]:
def Balancing(Data, classes, Buffer1, OriginalData):
    if len(np.where(classes == 1)[0]) == 0:
        idx  = np.random.choice(Buffer1)
        # row = OriginalData.iloc[idx, OriginalData.columns != 'Class']
        row = OriginalData.drop(OriginalData.columns[-1], axis=1).iloc[idx, :]

        # Class =  OriginalData.iloc[idx, OriginalData.columns == 'Class']
        Class =  OriginalData.iloc[idx, -1]

        index = np.random.choice(len(Data))
        Data[index] = row
        classes[index] = Class
    return Data, classes

In [47]:
def initlize(Data, classes):
    """
    Environment hyperparameters:
    """
    Features_Number = Data.shape[1]
    ACTION_SPACE_SIZE = 2
    print('Features_Number', Features_Number)

    """
    initlize environment:
    """
    k, o = 7,7
    OBSERVATION_SPACE_VALUES = (k,o)
    FeaturesSubspace = np.array([np.random.randint(0, ACTION_SPACE_SIZE) for i in range(Features_Number)])
    env = FeaturesSubspaceEnvironment(FeaturesSubspace, 0, Data,classes, OBSERVATION_SPACE_VALUES)
    env.State = env.get_statistics_state()

    """
    Rl hyperparameters & agents:
    """
    EPISODES = 100
    N = 2000 # the number of samples in each chunk.
    MIN_IN_CHUNK = 20
    slice_size = 500
    TRAINABLE_CHUNK_SIZE = 1000
    agents = [DQNAgent(OBSERVATION_SPACE_VALUES) for i in range(Features_Number)]
    '''
    Exploration settings
    '''
    epsilon = 1  # not a constant, going to be decayed
    EPSILON_DECAY = 0.99
    MIN_EPSILON = 0.001

    '''
    More needed data structures:
    '''
    Results = {}

    return  Features_Number, ACTION_SPACE_SIZE, env, \
            EPISODES, agents, epsilon, EPSILON_DECAY, MIN_EPSILON, Results, N, TRAINABLE_CHUNK_SIZE,slice_size

# **Model training**

In [48]:
clear_session()

In [49]:
import multiprocessing as mp
import time

def model_training(Data, classes, cols, Buffer_Class1 , dataset):
      Features_Number, ACTION_SPACE_SIZE, env, EPISODES, \
      agents, epsilon, EPSILON_DECAY, MIN_EPSILON, Results, N, MIN_IN_CHUNK = initlize(Data, classes)
      step, train_rate = 50, 2
      Reward_model = Reward()
      for idx, samples in enumerate(range(0,dataset.shape[0], N)):

        if np.mod(idx, step) == 0 and idx > 0:
          train_rate += 1

        chunk_dataset = dataset[samples:min(samples+N, dataset.shape[0])]
        # Data = np.array(chunk_dataset.loc[:, chunk_dataset.columns != 'Class'])
        Data = np.array(dataset.drop(dataset.columns[-1], axis=1))
        # classes = np.array(chunk_dataset['Class'])
        classes =  np.array(dataset.iloc[:, -1])

        print('for chunck ' , idx)

        sliceSize = max(MIN_IN_CHUNK, int(np.ceil(len(Data)/EPISODES)))

        if np.mod(idx, train_rate) == 0:
           print('chunk', idx, 'is training...')

        for episode in tqdm(range(EPISODES)):

            # every episode we choose a different set of data.
            selected_rows = np.random.choice(len(Data), sliceSize)

            #print(selected_rows)

            env.Data = Data[selected_rows]
            env.classes = classes[selected_rows]

            env.Data, env.classes = Balancing(env.Data, env.classes, Buffer_Class1,dataset)
            cnt = 0

            if sum(env.SPACE == 1) == 0: # all 0? not valid space.
              env.selected = np.random.choice(len(env.SPACE))
              env.step(1)
              env.assign_new_stata()

            current_state = copy.deepcopy(env.State)
            current_Subspace = copy.deepcopy(env.SPACE)

            for agent in agents:
              env.selected = cnt
              # epsilon starts big, and decreases.. at the begining we go random .. then predict from the model.
              if np.random.random() > epsilon:
                  # Get action from Q table
                  action = np.argmax(agent.get_qs(current_state))
              else:
                  # Get random action
                  action = np.random.randint(0, env.ACTION_SPACE_SIZE)

              env.step(action) #important for changeing the space.
              cnt = cnt + 1 ## the features counter.

            if sum(env.SPACE == 1) == 0: ##not valid.
              env.selected = np.random.choice(len(env.SPACE))
              env.step(1) #important for changeing the space. :)

            #after changing the whole space, we change the state...
            env.assign_new_stata()

            selectedSubspace = copy.deepcopy(env.SPACE)
            env.accuracy = F_score(Data[:, np.where(env.SPACE)[0]],classes)

            reward = env.get_reward() + env.accuracy

            new_state = copy.deepcopy(env.State)
            cnt = 0
            for agent in agents:

                transition = (current_state, selectedSubspace[cnt], reward, new_state, selectedSubspace, current_Subspace)
                cnt = cnt + 1
                agent.update_replay_memory(transition)
                if np.mod(idx, train_rate) == 0:
                  agent.train()

            # Decay epsilon
            if epsilon > MIN_EPSILON:
                epsilon *= EPSILON_DECAY
                epsilon = max(MIN_EPSILON, epsilon)

        ## should be for the whole chunk.

        acc = F_score(Data[:, np.where(env.SPACE)[0]], classes)
        rd = Reward_model.GetMiRedundancy(Data[:, np.where(env.SPACE)[0]])
        rv = Reward_model.GetMiRelevance(Data[:, np.where(env.SPACE)[0]], classes)

        reward =  rv + acc - rd
        subspace = copy.deepcopy(env.SPACE)

        Results[idx] = (copy.deepcopy(np.where(env.SPACE)), (acc, rd, rv))

        if sum(subspace == 1) == 0:
          raise ValueError(
                  f"NOT Valid space",
          )

        print('Final chunk results:')
        print('reward' , reward)
        print('the selected features: ' , np.where(env.SPACE))
        print('****************Done chunck ' , idx , '*************************')
      return Results


# Pipeline:

In [50]:
#'IramTariq_Dmoz_xssed.csv' Done
# 'Data_167_featurs.csv' Done
# 'IramTariq_DataTakenFromYunZhou.csv' [0,1,2,3,4]
# 'Data_66_featurs.csv' Done
filenames = ['IramTariq_DataTakenFromYunZhou.csv']
filenames = list(map(lambda x: os.path.join(data_path, x), filenames))
filenames

['/content/drive/MyDrive/Colab Notebooks/Muawiya/Js_Contana/dynamic-feature-selection/Data/isam data/IramTariq_DataTakenFromYunZhou.csv']

In [51]:
def pipeline(data_url,reslts_path):
    os.makedirs(reslts_path, exist_ok=True)
    saved_data = os.path.join(reslts_path,'dataset.pkl')
    os.makedirs(reslts_path, exist_ok=True)
    dataset = read_preprocess(data_url, saved_data)
    Data, classes, cols, Buffer_Class1 = ExploreData(dataset)
    #Results = model_training(Data, classes, cols, Buffer_Class1 , dataset)
    return  Data, classes, cols, Buffer_Class1, dataset

In [52]:
for filename in tqdm(filenames):
  reslts_path = os.path.join(root_reslts_path,filename.split("/")[-1].split(".")[0])
  Data, classes, cols, Buffer_Class1,  dataset = pipeline(data_url = filename,reslts_path=reslts_path)

  # Features_Number, ACTION_SPACE_SIZE, _, \
  #             EPISODES, _, epsilon, EPSILON_DECAY, MIN_EPSILON, _, N, TRAINABLE_CHUNK_SIZE,slice_size = initlize(Data, classes)
  Features_Number, ACTION_SPACE_SIZE, env, \
              EPISODES, agents, epsilon, EPSILON_DECAY, MIN_EPSILON, Results, N, TRAINABLE_CHUNK_SIZE,slice_size = initlize(Data, classes)
  if dataset.shape[0]<20000:
    N=1000
  # the rate of training...
  trainable_chunk = pd.DataFrame()
  Reward_model = Reward()
  chunk_number = 0
  chunk_done = 4
  print(50*"*",filename.split("/")[-1].split(".")[0],50*"*")
  print("Total Number of chunk : ",dataset.shape[0]/N)
  for idx, samples in enumerate(range(0,dataset.shape[0], N)):

      if chunk_number <= chunk_done and filename.split("/")[-1].split(".")[0] == 'IramTariq_DataTakenFromYunZhou' :
        env = load_object('env_statistics',reslts_path)
        agents = load_object('agents_statistics',reslts_path)
        dataset = load_object('dataset_statistics',reslts_path)
        Results = load_object('results_statistics',reslts_path)
        # epsilon = load_object('epsilons_statistics',reslts_path)
        chunk_number+=1
        continue

      chunk_dataset = dataset[samples:min(samples+N, dataset.shape[0])]
      trainable_chunk = pd.concat([trainable_chunk, chunk_dataset])
      #print('for chunck ' , idx)

      if idx  == 0:
          Data = np.array(trainable_chunk.drop(trainable_chunk.columns[-1], axis=1)) #np.array(trainable_chunk.loc[:, trainable_chunk.columns != 'Class'])
          classes = np.array(trainable_chunk.iloc[:, -1]) #np.array(trainable_chunk['Class'])

          Data_after_selection = Data[:, np.where(env.SPACE)[0]]
          acc = F_score(Data_after_selection, classes)
          rd = Reward_model.GetMiRedundancy(Data_after_selection)
          rv = Reward_model.GetMiRelevance(Data_after_selection, classes)

          reward =  rv + acc - rd
          Results[idx] = (copy.deepcopy(np.where(env.SPACE)), (acc, rd, rv, reward))

      if len(trainable_chunk) >= TRAINABLE_CHUNK_SIZE:
          #print('training time...' , idx)
          #print(trainable_chunk.shape)
          Data = np.array(trainable_chunk.drop(trainable_chunk.columns[-1], axis=1)) #np.array(trainable_chunk.loc[:, trainable_chunk.columns != 'Class'])
          classes = np.array(trainable_chunk.iloc[:, -1]) #np.array(trainable_chunk['Class'])

          for episode in tqdm(range(EPISODES)):

              # every episode we choose a different set of data.
              selected_rows = np.random.choice(len(Data), slice_size)

              env.Data = Data[selected_rows]
              env.classes = classes[selected_rows]

              env.Data, env.classes = Balancing(env.Data, env.classes, Buffer_Class1,dataset)
              cnt = 0

              current_state = copy.deepcopy(env.State)
              current_Subspace = copy.deepcopy(env.SPACE)

              for agent in agents:
                  env.selected = cnt
                  # epsilon starts big, and decreases.. at the begining we go random .. then predict from the model.
                  if np.random.random() > epsilon:
                      # Get action from Q table
                      action = np.argmax(agent.get_qs(current_state))
                  else:
                      # Get random action
                      action = np.random.randint(0, env.ACTION_SPACE_SIZE)

                  env.step(action) #important for changeing the space.
                  cnt = cnt + 1 ## the features counter.

              if sum(env.SPACE == 1) == 0:
                print('failed chunk..' , idx)
                raise ValueError("IN 0 state")

              #after changing the whole space, we change the state...
              env.assign_new_stata()

              selectedSubspace = copy.deepcopy(env.SPACE)
              env.accuracy = F_score(Data[:, np.where(env.SPACE)[0]],classes)
              reward = env.get_reward() + env.accuracy

              new_state = copy.deepcopy(env.State)
              cnt = 0
              for agent in agents:

                  transition = (current_state, selectedSubspace[cnt], reward, new_state, selectedSubspace, current_Subspace)
                  cnt = cnt + 1
                  agent.update_replay_memory(transition)


              # Decay epsilon
              if epsilon > MIN_EPSILON:
                  epsilon *= EPSILON_DECAY
                  epsilon = max(MIN_EPSILON, epsilon)

          ## should be for the whole chunk.
          for agent in agents:
              agent.train()

          Data_after_selection = Data[:, np.where(env.SPACE)[0]]
          acc = F_score(Data_after_selection, classes)
          rd = Reward_model.GetMiRedundancy(Data_after_selection)
          rv = Reward_model.GetMiRelevance(Data_after_selection, classes)

          reward =  rv + acc - rd
          subspace = copy.deepcopy(env.SPACE)

          Results[idx] = (copy.deepcopy(np.where(env.SPACE)), (acc, rd, rv, reward))

          if sum(subspace == 1) == 0:
              raise ValueError(f"NOT Valid space",)
          print('Final chunk results:')
          print('reward, acc, rd, rv' , reward , acc, rd, rv)
          print('the selected features: ' , np.where(env.SPACE))
          print('****************Done chunck ' , idx , chunk_number, '*************************')
          save_object(env, 'env_statistics',reslts_path)
          save_object(agents, 'agents_statistics',reslts_path)
          save_object(dataset, 'dataset_statistics',reslts_path)
          save_object(Results, 'results_statistics',reslts_path)
          save_object(epsilon,'epsilons_statistics',reslts_path)

          trainable_chunk = pd.DataFrame()
      chunk_number+=1

  0%|          | 0/1 [00:00<?, ?it/s]

0.6503 0.3497
Features_Number 30
************************************************** IramTariq_DataTakenFromYunZhou **************************************************
Total Number of chunk :  10.0


  0%|          | 0/100 [00:00<?, ?it/s]

Final chunk results:
reward, acc, rd, rv 1.1442392340203718 0.9949894724697873 0.015481119936662104 0.16473088148724643
the selected features:  (array([ 0,  1,  3,  4,  5,  6,  7,  8,  9, 12, 14, 15, 16, 21, 23, 24, 26,
       28, 29]),)
****************Done chunck  5 5 *************************


  0%|          | 0/100 [00:00<?, ?it/s]

...

Final chunk results:
reward, acc, rd, rv 1.1531626290112746 0.99 0.018013816793868796 0.18117644580514353
the selected features:  (array([ 0,  1,  2,  3,  4,  5,  7,  8, 12, 13, 14, 18, 20, 21, 22, 24, 26,
       28]),)
****************Done chunck  7 7 *************************


  0%|          | 0/100 [00:00<?, ?it/s]

Final chunk results:
reward, acc, rd, rv 1.133599989446971 0.989974533106961 0.014992921178593688 0.15861837751860372
the selected features:  (array([ 0,  1,  3,  4,  5,  6,  7,  8, 12, 13, 14, 18, 20, 21, 22, 23, 24,
       26, 28]),)
****************Done chunck  8 8 *************************


  0%|          | 0/100 [00:00<?, ?it/s]

Final chunk results:
reward, acc, rd, rv 1.1322188305113638 0.99 0.013928747277153496 0.15614757778851732
the selected features:  (array([ 0,  1,  3,  4,  5,  6,  7,  8, 12, 13, 14, 18, 20, 21, 22, 23, 24,
       26, 28]),)
****************Done chunck  9 9 *************************


# Saving